In [1]:
import pandas as pd
rsvp = pd.read_excel('rsvp.xlsx')
member = pd.read_csv('members.csv',encoding='latin-1')
events = pd.read_csv('events.csv',encoding='latin-1')

In [25]:
rsvp.head()

,Unnamed: 0,event_id,member_id
0,0,cqkdxnywnbpc,12067672
1,1,cqkdxnywnbpc,234733619
2,2,cqkdxnywnbpc,78357452
3,3,cqkdxnywnbpc,224346861
4,4,cqkdxnywnbpc,239059586


In [3]:
member.head(2)

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
0,3,not_found,New York,us,"New York, NY",2007-05-01 22:04:37,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2009-09-18 18:32:23,490552
1,3,not_found,New York,us,"New York, NY",2011-01-23 14:13:17,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-03-20 01:02:11,1474611


In [4]:
events.head(2)

,event_id,created,description,duration,event_url,fee.accepts,fee.amount,fee.currency,fee.description,fee.label,...,venue.lon,venue.name,venue.phone,venue.repinned,venue.state,venue.zip,visibility,waitlist_count,why,yes_rsvp_count
0,153868222,2013-12-03 21:24:29,Solve the murder case while eating a 3 course ...,20700,https://www.meetup.com/SanFranciscoStartupFun/...,others,0.0,not_found,per person,Price,...,-122.405690,Little Szechuan,-1,1,CA,-1,public,0,not_found,72
1,184167702,2014-05-20 18:52:00,NOTE: This event is EVERY FRIDAY!! Signup is a...,77400,https://www.meetup.com/hr-and-tech-sf/events/1...,others,0.0,not_found,per person,Price,...,-122.405457,Horizon Lounge,-1,0,CA,-1,public,0,not_found,179


In [5]:
#preprocesssing
main_data = pd.merge(rsvp, events, on= 'event_id', how='inner') ; main_data.head()
temp_data = main_data[['event_id', 'member_id', 'group_id', 'group.name']] ; temp_data.head()
main_data_2 =temp_data.groupby(['group_id','member_id']).size().reset_index() ; print (main_data_2.head())
print (main_data_2.shape) ; print (main_data_2.member_id.nunique())
main_data_2.columns = ['group_id', 'member_id','rsvp_sum']#shows how many events in a group a member has rsvp'ed to

   group_id  member_id   0
0    413427    5033164   1
1    413427   12142571   1
2    413427   31492942  12
3    413427   36962882   1
4    413427   51417262   1
(3039, 3)
2958


In [6]:
#standardizing to 1 - 10 scale 
main_data_2['rsvp_total'] = 1 + (main_data_2['rsvp_sum']-main_data_2['rsvp_sum'].min())*(10-1)/(main_data_2['rsvp_sum'].max()\
                                                                                                -main_data_2['rsvp_sum'].min())

In [7]:
main_data_2.sort_values(by='rsvp_total',ascending=False)
data=main_data_2[['group_id','member_id','rsvp_total']]

In [8]:
data = main_data_2.dropna()
data = data.copy()

# Converting the numbers to categories to be used for creating the categorical codes to avoid using long hash keys 
data['member_id'] = data['member_id'].astype('category')
data['group_id'] = data['group_id'].astype('category')

#cat.codes creates a categorical id for the users and artists
data['user_id'] = data['member_id'].cat.codes
data['grp_id'] = data['group_id'].cat.codes

In [9]:
users=data[['user_id','grp_id','rsvp_total']]

User based collaborative filtering

In [11]:
#Create list of groups for each user
users_groups=users.groupby('user_id')['grp_id'].apply(list)
users_groupslist=users_groups.to_list()

In [10]:
#Finding cosine similarity between two vectors
def dot(v,w):
    """ v_1*w_1 + ... v_n*w_n"""
    return sum(v_i *w_i
              for v_i,w_i in zip(v,w))
def cosine_similarity(v, w):
    return dot(v, w) / math.sqrt(dot(v, v) * dot(w, w))

In [12]:
#Creating user-rsvp matrix for all groups
users_df = pd.pivot_table(users, index='user_id', columns='grp_id', values='rsvp_total')
users_df.fillna(0,inplace=True)
users_df=pd.DataFrame(users_df)
users_df.shape

(2958, 47)

In [42]:
#Converting dataframe to list of lists
users_df = users_df.rename_axis(None, axis=1)
users_list=users_df.values.tolist()

In [22]:
#Finding cosine similarity between users
import math
user_similarities = [[cosine_similarity(group_vector_i, group_vector_j)
                      for group_vector_j in users_list]
                     for group_vector_i in users_list]

In [23]:
#Finding most similar users
def most_similar_users_to(user_id):
    pairs = [(other_user_id, similarity)              
             for other_user_id, similarity in         
                enumerate(user_similarities[user_id]) 
             if user_id != other_user_id and similarity > 0]

    return sorted(pairs,                              
                  key=lambda pair: pair[1],           
                  reverse=True) 

In [224]:
#Finding group suggestions based on similar users
from collections import defaultdict, Counter
def user_based_suggestions(user_id, include_current_groups=True):
    suggestions = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for groups in users_groupslist[other_user_id]:
            suggestions[groups] += similarity

    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)
    
    if include_current_groups:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                    if suggestion not in users_groupslist[user_id]]

In [243]:
user_based_suggestions(1429)

[(35, 100.09352357598202),
 (11, 2.227053698519673),
 (40, 2.116344055900802),
 (18, 0.7501258215615596)]

In [244]:
#Finding top groups for each user
top_tuple=user_based_suggestions(1429)[0:5]
top_list=[x[0] for x in top_tuple]

In [245]:
top_groups=data[data['grp_id'].isin(top_list)]['group_id'].unique()
top_groupnames=events[events['group_id'].isin(top_groups)]['group.name']
top_groupnames=top_groupnames.unique()

In [246]:
print("Recommendations based on user based collaborative filtering:")
for i in range(len(top_groupnames)):
    print(top_groupnames[i])

Recommendations based on user based collaborative filtering:
Women Organize For Success - Meetup
Lexington/Vanderbilt toastmasters public speaking Meetup
Bryant Park Toastmasters
SEO and Social Media Marketing Academy


Item based collaborative filtering

In [14]:
#Creating group-rating matrix for all users
groups_list_new=[]
groups_df = pd.pivot_table(users, index='grp_id', columns='user_id', values='rsvp_total')
groups_df.fillna(0,inplace=True)
groups_df=pd.DataFrame(groups_df)
groups_df = groups_df.rename_axis(None, axis=1)
groups_list=groups_df.values.tolist()

In [15]:
#Creating complete list of unique groups across all users
flat_list = [item for sublist in users_groupslist for item in sublist]
list_set = set(flat_list) 
unique_list = (list(list_set)) 

In [47]:
import math
#Finding cosine similarity between movies
group_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                          for user_vector_j in groups_list]
                          for user_vector_i in groups_list]

In [48]:
#Finding most similar groups
def most_similar_groups_to(group_id):
    similarities = group_similarities[group_id]
    pairs = [(unique_list[other_group_id], similarity)
             for other_group_id, similarity in enumerate(similarities)
             if group_id != other_group_id and similarity > 0]
    return sorted(pairs,
                  key=lambda pair: pair[1],
                  reverse=True)

In [49]:
#Finding group suggestions
from collections import defaultdict, Counter
def group_based_suggestions(user_id, include_current_groups=True):
    suggestions = defaultdict(float)
    user_vector=users_list[user_id]
    for group_id, rsvp_count in enumerate(user_vector):
        if rsvp_count!=0:
            similar_groups=most_similar_groups_to(group_id)
            for groups,similarity in similar_groups:
                suggestions[groups] += similarity

    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)
    
    if include_current_groups:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                    if suggestion not in users_groupslist[user_id]]

In [50]:
#Finding top 5 movie names from movies dataframe
top_tuple=group_based_suggestions(1429)[0:5]
top_list=[x[0] for x in top_tuple]
top_groups=data[data['grp_id'].isin(top_list)]['group_id'].unique()
top_groupnames=events[events['group_id'].isin(top_groups)]['group.name']
top_groupnames=top_groupnames.unique()

In [51]:
print("Recommendations based on item based collaborative filtering:")
for i in range(len(top_groupnames)):
    print(top_groupnames[i])

Recommendations based on item based collaborative filtering:
Women Organize For Success - Meetup
Lexington/Vanderbilt toastmasters public speaking Meetup
SEO and Social Media Marketing Academy
